In [1]:
import pandas as pd

df= pd.read_excel('/home/arspraxia/Documents/Json Control Final/source/JSONconverter/transformed_20231110_원천데이터(기사)_19.xlsx')

In [2]:
tags_list = df.iloc[73191, 13:]
first_row=df.iloc[73188, 13:len(tags_list)].dropna()

In [16]:
first_row = df.iloc[73188].dropna()
first_row= first_row.replace(r'^\s*$', "'", regex=True)

In [19]:
first_row[13:].to_list()

["'",
 'Menurut',
 'orang',
 'dalam',
 'industri',
 'hiburan',
 ',',
 'B1A4',
 'berencana',
 'untuk',
 'merilis',
 'single',
 'Natal',
 'pada',
 '10',
 'Desember',
 '.']

In [15]:
" ".join(eval(first_row["Token"]))

KeyError: 'Token'

In [ ]:
if "O" not in tags_list.tolist():
    for tag_idx, col in enumerate(first_row[14:]):
        if pd.isna(tags_list.iloc[tag_idx]):
            tags_list.iloc[tag_idx] = "O"
tags_list = tags_list.dropna().tolist()

In [9]:
tags_list = df.iloc[73191 + 1, 13:]first_row = df.iloc[i].fillna("'")
test=df.iloc[73190, 13:len(tags_list)].fillna("'")

In [7]:
print(test)

Doc_ID                          20231009_newsdata_Jepang_003809
Filename      C:/Users/USER/Desktop/기사 데이터_로컬환경/20231009_dat...
Title                     B1A4 Bersiap untuk Rilis Single Natal
Pub_Type                                              Newspaper
Pub_Subj                                                 Jepang
                                    ...                        
Column_487                                                    '
Column_488                                                    '
Column_489                                                    '
Column_490                                                    '
Column_491                                                    '
Name: 73190, Length: 505, dtype: object


In [ ]:
df['Doc']

In [ ]:
for i in range(0, len(df), 2):
    # 2행씩 묶어서 처리
    # 두 번째 행의 열 중 Column2(인덱스 14)부터 끝까지 가져옴
    # nan 제외하고 가져오기
    # 이 정보를 리스트로 변환
    try:
        # first_row = df.iloc[i].dropna()
        first_row = df.iloc[i, len()].fillna("'")
        token_list = df.iloc[i, 13:]
        tags_list = df.iloc[i + 1, 13:]
    except IndexError:
    # 정확한 원인은 모르겠으나, IndexError가 발생하는 경우가 있음. 이런 때 해당 파일은 마지막 부분이므로 건너뜀
        break

In [ ]:
first_ro

# Split train/validiation/test

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 기본 폴더 경로 설정
base_folder = '/home/arspraxia/ner-training/dataset/'
source_folders = [ '03.Extra']
target_folders = ['train', 'validation', 'test']

# Twitter 폴더들을 하나로 통합 처리하기 위한 매핑
folder_mapping = {
    '02.Twitter': '02.Twitter',
    '02.Twitter_2': '02.Twitter'
}

# train, validation, test 폴더 생성
for target_folder in target_folders:
    target_path = os.path.join(base_folder, target_folder)
    if not os.path.exists(target_path):
        os.makedirs(target_path)

# 각 소스 폴더 순회
for source_folder in source_folders:
    folder_path = os.path.join(base_folder, source_folder)
    all_files = os.listdir(folder_path)
    json_files = [f for f in all_files if f.endswith('.json')]

    # 파일 목록을 무작위로 섞고 8:1:1 비율로 분할
    train_files, test_files = train_test_split(json_files, test_size=0.2, random_state=42)
    validation_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

    # 폴더 매핑에 따라 대상 폴더 결정
    target_source_folder = folder_mapping.get(source_folder, source_folder)

    # 분할된 파일을 각각의 폴더로 복사
    for target_folder, files in zip(target_folders, [train_files, validation_files, test_files]):
        target_dir_path = os.path.join(base_folder, target_folder, target_source_folder)
        if not os.path.exists(target_dir_path):
            os.makedirs(target_dir_path)
        # Progress bar 추가
        for file in tqdm(files, desc=f"Copying files to {target_folder}/{target_source_folder}"):
            source_file_path = os.path.join(folder_path, file)
            shutil.copy(source_file_path, target_dir_path)

print("파일 분할 및 복사가 완료되었습니다.")


Copying files to test/03.Extra: 100%|██████████| 3273/3273 [00:00<00:00, 4440.80it/s]

파일 분할 및 복사가 완료되었습니다.


# Count Pub_Subj

In [20]:
import os
import pandas as pd
from tqdm import tqdm

# 두 폴더 경로 설정
folders = [
    '/home/arspraxia/ner-training/dataset/02.Twitter',
    '/home/arspraxia/ner-training/dataset/02.Twitter_2'
]

# 주제어를 카운트할 빈 딕셔너리 초기화
topic_counts = {}

# 두 폴더 모두 순회
for folder_path in folders:
    # 폴더 내의 모든 파일 순회, progress bar 표시
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {folder_path}"):
        if filename.endswith('.json'):  # JSON 파일인지 확인
            # 파일명에서 주제어 추출 ('_'를 기준으로 분리 후 원하는 부분 선택)
            parts = filename.split('_')
            if len(parts) > 2:  # 파일명이 예상 구조와 일치하는지 확인
                topic = parts[2]  # 예시에서 "Bali"에 해당하는 부분
                # 주제어별 카운트 수행
                if topic in topic_counts:
                    topic_counts[topic] += 1
                else:
                    topic_counts[topic] = 1

# 딕셔너리를 데이터 프레임으로 변환
df = pd.DataFrame(list(topic_counts.items()), columns=['Topic', 'Count'])

# 데이터 프레임 출력
print(df)


Processing /home/arspraxia/ner-training/dataset/02.Twitter: 100%|██████████| 5000000/5000000 [00:02<00:00, 2166260.70it/s]
Processing /home/arspraxia/ner-training/dataset/02.Twitter_2: 100%|██████████| 3430099/3430099 [00:01<00:00, 2162388.37it/s]

        Topic    Count
0      kdrama  1061798
1   Seventeen   157530
2       Korea  3974053
3        Yoon    90428
4    newjeans    76233
..        ...      ...
56        Kep        1
57    Samsung  1441518
58       Suzy    11213
59   Mabinogi      245
60  Shinsegae       13

[61 rows x 2 columns]


In [25]:
df['Count'].sum()

8430099

In [26]:
df.to_csv('99.Twitter 검색어 통계.csv', index=False)

In [2]:
import unicodedata

def remove_diacritics(input_str):
    # Normalize the input string to decompose characters with diacritics
    normalized = unicodedata.normalize('NFD', input_str)
    # Filter out characters that are not in the basic Latin alphabet (i.e., remove diacritics)
    no_diacritics = ''.join(char for char in normalized if unicodedata.category(char) != 'Mn')
    return no_diacritics

# Example usage
input_str = """
Università degli Studi di Firenze è un'istituzione che valorizza la ricerca e l'innovazione.
Ça va bien, merci. Cómo estás? Muy bien, gracias.
Große Straße, Ärger, Ökonomie, Übung, Łódź, ślub, život, čas, đất, țară, ūdens, ğöğüs.
Façade, naïve, coöperate, São Paulo, Señor, piña colada, élève, crème brûlée, fiancée.
"""
output_str = remove_diacritics(input_str)
print(output_str)  # Output: Cafe, facade, resume, naive, cooperation



Universita degli Studi di Firenze e un'istituzione che valorizza la ricerca e l'innovazione.
Ca va bien, merci. Como estas? Muy bien, gracias.
Große Straße, Arger, Okonomie, Ubung, Łodz, slub, zivot, cas, đat, tara, udens, gogus.
Facade, naive, cooperate, Sao Paulo, Senor, pina colada, eleve, creme brulee, fiancee.



In [4]:
from pathlib import Path
from tool import handler1221

source_path = Path("/home/arspraxia/Documents/Handled_Data4/wiki/")
result_path = Path("result_storage/wiki_handle_test/")

handler1221.main(
    source_path, result_path,
    report_interval=1000
)

13:14:14	 0:00:00 passed	task_count: 0 

13:14:16	 0:00:02 passed	task_count: 1000 

13:14:18	 0:00:02 passed	task_count: 2000 

13:14:21	 0:00:02 passed	task_count: 3000 

13:14:23	 0:00:02 passed	task_count: 4000 



OSError: [Errno 36] File name too long: 'result_storage/wiki_handle_test/wiki/20230816_wikidata_Kementerian Perencanaan Pembangunan Nasional Republik Indonesia_Kementerian Perencanaan Pembangunan Nasional Republik Indonesia_Kementerian Perencanaan Pembangunan Nasional Republik Indonesia_Badan Perencanaan Pembangunan Nasional_0000001.json'

In [2]:
from tool.JSONformat_handler import remove_duplicates_keep_order

"_".join(remove_duplicates_keep_order(
"20230816_wikidata_wikipedia_artikel bagus_peninjauan ulang_wikipedia:artikel bagus_wikipedia:artikel bagus_peninjauan ulang_korea selatan_0000001".split("_")))

'20230816_wikidata_wikipedia_artikel bagus_peninjauan ulang_wikipedia:artikel bagus_korea selatan_0000001'

In [3]:
from tool.JSONformat_handler import remove_diacritics, split_latin_ligatures, normalize_string
import json

with open("/home/arspraxia/Documents/Json Control Final/result_storage/wiki_handle_test/wiki/20230816_wikidata_oh yeon-seo_0000001.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

data_string = json.dumps(data, ensure_ascii=False)
data_string_rm = remove_diacritics(data_string)
data_string_sll = split_latin_ligatures(data_string_rm)
new_data = json.loads(data_string_sll)

In [4]:
new_data

{'Doc_ID': '20230816_wikidata_oh yeon-seo_0000001',
 'Filename': 'C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NIA AI 데이터셋 구축 - General/06-수집데이터/위키 데이터/rawdata/idwiki-20230720-pages-articles-multistream.xml',
 'Title': 'Oh Yeon-seo',
 'Text': 'Oh Yeonseo , adalah seorang aktris, penyanyi dan model Korea Selatan dan terkenal karena perannya dalam drama televisi My Husband Got a Family , Jang Bori is Here , Shine or Go Crazy , Come Back Mister , My Sassy Girl , A Korean Odyssey , Love with Flaws , Mad for Each Other , dan Cafe Minamdang .  Oh Yeonseo lahir di Jinju, Provinsi Gyeongsang Selatan dan dibesarkan di Kabupaten Changnyeong. Di tahun kedua sekolah menengah, Oh mengikuti sekelompok temannya untuk mengikuti audisi SM Entertainment, yang diadakan di Daegu, tetapi ditolak. Namun, perusahaan lain menghubunginya mengatakan bahwa dia telah memenuhi syarat dan dia pindah ke Seoul, memulai debutnya empat bulan kemudian pada usia 15 tahun. Oh Yeonseo diterima di Anyang Arts High School set